<a href="https://colab.research.google.com/github/Victhagas/BCI/blob/main/CyberLab_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Desafio! Criar um classificador que consiga diferenciar os números 0 e 5

!pip install -Uqq fastbook

import fastbook
import numpy             as np
import pandas            as pd
import tensorflow        as tf
import matplotlib.pyplot as plt

from fastai.vision.all import *
from fastbook          import *

fastbook.setup_book()
matplotlib.rc('image', cmap='Greys')

In [ ]:
# Pegar os arquivos de imagem correspondentes aos números 0 e 5 do dataset MNIST.
# Getting the image files corresponding to the numbers 0 and 5 from the MNIST dataset.

path = untar_data(URLs.MNIST) # Como o arquivo original do Dataset é comprimido, é necessário usar o untar_data.
Path.BASE_PATH = path         # As the original Dataset file is compressed, it is necessary to use untar_data.

In [ ]:
path.ls() # Usar o .ls para ver os dados contidos no dataset MNIST.
          # Using .ls to see the data from the MNIST dataset.

In [ ]:
(path/'training/0').ls() # Especificando o numeral da busca no .ls
                         # Specifying the numeral which we are searching for, in .ls

In [ ]:
zeroes_train = (path/'training/0').ls() # Armazenando os dados de treino correspondentes ao numeral em uma variável (i.e: zero)
fives_train  = (path/'training/5').ls() # Storing the training data corresponding to the numeral in a variable (i.e: five)

zeroes_test = (path/'testing/0').ls()   # Mesmo caso das variáveis acima, porém com os dados de teste
fives_test  = (path/'testing/5').ls()   # Same thing as above, but using the variable to save the testing data 

In [ ]:
train_0 = torch.stack([tensor(Image.open(i)) for i in zeroes_train]).float()/255 # list comprehension para transformar todas as imagens de treino em 
# tensores. Após isso, juntar todos os novos tensores em um só (torch.stack). Por fim, deixar cada pixel desse novo tensor em escala 0~1. 

train_5 = torch.stack([tensor(Image.open(i)) for i in fives_train ]).float()/255 # list comprehension to transform all training images into
# tensors. After that, join all the new tensors into one (torch.stack). Finally, let each pixel of this new tensor in 0~1 scale. 

test_0  = torch.stack([tensor(Image.open(i)) for i in zeroes_test ]).float()/255 # Mesma ideia da variável anterior, porém com os as imagens de teste 

test_5  = torch.stack([tensor(Image.open(i)) for i in fives_test  ]).float()/255 # Same idea from above, but using the testing images. 

print(train_0.shape, train_5.shape) # Checando os tamanhos das nossas novas variaveis! 
print(test_0.shape, test_5.shape);  # Checking the shapes of our new born variables!

In [ ]:
train_x = torch.cat([train_0, train_5]).view(-1, 28*28) # Concatenando as duas variáveis de treino e mudando o tamanho do tensor.
train_y = tensor([1]*len(zeroes_train) + [0]*len(fives_train)).unsqueeze(1) # Rotulando cada imagem do dataset e modelando o tensor

dset = list(zip(train_x,train_y)) # Criando um dataset combinando a função zip + list, nos dados de treino.
x,y  = dset[0]

test_x    = torch.cat([test_0, test_5]).view(-1, 28*28) # Concatenating the two training variables and changing the tensor's shape.
test_y    = tensor([1]*len(test_0) + [0]*len(test_5)).unsqueeze(1) # Labeling each image and reshaping the tensor

test_dset = list(zip(test_x,test_y)) # Creating a dataset, by combining zip and list functions, in the testing data. 

train_x.shape,train_y.shape # Checking some of our new tensors!!

In [ ]:
def _parameters(size, std=1.0): # The first step to initialize the weights, randomly, for every pixel.
                                # The first step to initialize the weights, randomly, for every pixel.

  return (torch.randn(size)*std).requires_grad_()

def mult_mat(xn): # The fundamental equation, batch*weights + bias. With * being a matricial multiplication.
                  # The fundamental equation, batch*weights + bias. With * being a matricial multiplication.
  
  return xn@weights + bias
  
def loss_function(predictions, targets): # Essa função vai medir a distancia entre o valor da predição do seu alvo, e limitar a predição à ser entre 0 e 1
    predictions = predictions.sigmoid()  # This function will measure the distance of the predictions from the real values, and limit the predictions (0~1)
    
    return torch.where(targets==1, 1-predictions, predictions).mean() # Medindo a distancia da predição ao seu alvo.    
                                                                      # Measuring the distance of the predictions from its target.

def calc_grad(xn, yn, model): # Como o nome diz, essa função é responsavel por calcular o gradiente, e para isso, utiliza-se a loss function.
    preds = model(xn)         # As the name says, this definition will calculate the gradient value, by using the loss function.
    loss  = loss_function(preds, yn) 
    loss.backward()           # loss.backward() calcula a taxa de variação (dloss/dx) para cada parametro em x. 
                              # loss.backward() computes dloss/dx for every parameter x.

weights = _parameters((28*28,1)) # Inicializando os Pesos (weights), que irão de fato fazer o aprendizado de máquina 
bias    = _parameters(1)         # Initializing the bias, which are going to do the machine learning.

dload      = DataLoader(     dset, batch_size=256) # O DataLoades promove uma iteração entre o dataset predefinido e as batches.
test_dload = DataLoader(test_dset, batch_size=256) # the DataLoader configurs an interation of the dataset over the batches.

batch   = train_x[:4]       # Tamanho das mini-batches.
                            # Size of the mini-batches.

predict = mult_mat(batch)   # Usando a definição da função fundamental com a batch setada acima como input.
                            # Using the definition of the fundamental function, with the batch set above as input.

loss    = loss_function(predict, train_y[:4]) # Armazenando a definição loss_function já com inputs especificos em uma variável, para facilitar.
                                              # Saving the loss_function definition in a variable, to make things easier.

xn,yn   = first(dload)

In [ ]:
def training_epochs(model, learn_r, _parameters): # Definição que de fato faz o treinamento, usando todos os parâmetros setados acima
                                                  # Definition that does the training over the epochs, by using all parameters set above

    for xn,yn in dload:                           
        calc_grad(xn, yn, model)
        for i in _parameters:
            i.data = i.data - i.grad*learn_r # Dentro da iteração no DataLoad, ocorre outra iteração que atualiza os dados usando learning ratio.
                                             # Inside the DataLoad for loop, occurs other iteration loop that update the data by using the learning ratio.
            i.grad.zero_()                   

def batch_acc(xn, yn):                       # Definição para corrigir e efetuar a média das predições, além de limitar o valor entre 0 e 1.
                                             # Definition that corrects and does the mean of predictions, and fix its limits between 0 and 1.
    preds   = xn.sigmoid()                    
    correct = (preds>0.5) == yn
    return correct.float().mean()

def testing_epochs(model):                    # Definição para juntar a batch_acc (definição acima) para todas as batches.
                                              # Definition that gather all batches accuracy.
    accs = [batch_acc(model(xn), yn) for xn,yn in test_dload]
    return round(torch.stack(accs).mean().item(), 4)
  
testing_epochs(mult_mat)

In [ ]:
learn_r = 1.  
_parameters = weights,bias

for i in range(30):                               # Setando a quantidade de sessões de treinamento o classificador receberá.
                                                  # For loop to set the training sections' quantity the classifier will receive.
  training_epochs(mult_mat,learn_r,_parameters)   
  print(testing_epochs(mult_mat), end=' ')
testing_epochs(mult_mat)